In [2]:
# isntall libraries
!pip install requests pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# P1. I first explore these 3 platforms.

## aave
This API is free to use.
Here we can fetch
- Tvl: in JSON format
- Liquidity: in HTML format
- staking pool status: in JSON format

We do not fetch anything with user or address

API documentation: https://aave-api-v2.aave.com/#/data/get_data_liquidity_v1

In [ ]:
# save the data 
import requests
import json
import pandas as pd

BASE_URL = "https://aave-api-v2.aave.com"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Save data to CSV
def save_to_csv(data, filename):
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

# Flatten nested dictionary for easier CSV storage
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


In [2]:
# 1. Total Value Locked (TVL) - updated every 15 minutes
def get_tvl():
    url = f"{BASE_URL}/data/tvl"
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            print("Total Value Locked (TVL):")
            print(json.dumps(data, indent=4))
            flat_data = flatten_dict(data)
            save_to_csv([flat_data], "aave_tvl.csv")
        else:
            print(f"Error fetching TVL: {response.status_code}")
            print(f"Details: {response.text}")
    except Exception as e:
        print(f"Failed to fetch TVL: {e}")
        
get_tvl()

Total Value Locked (TVL):
{
    "totalTvl": {
        "tvlInEth": "429994.10201757966",
        "tvlInUsd": "1332817296.237194"
    },
    "marketTvls": {
        "v1": {
            "tvlInEth": "4408.385052074143",
            "tvlInUsd": "13665932.464228552"
        },
        "staking": {
            "stkAave": {
                "tvlInEth": "156930.78142040712",
                "tvlInUsd": "486483243.89015436"
            },
            "stkABPT": {
                "tvlInEth": "561.9377175161609",
                "tvlInUsd": "1741999.1234807044"
            }
        },
        "v2": {
            "tvlInEth": "244985.0066771057",
            "tvlInUsd": "759450119.8171649"
        },
        "AMM": {
            "tvlInEth": "287.32070100901916",
            "tvlInUsd": "890690.1845419881"
        },
        "matic": {
            "tvlInEth": "22820.6704494675",
            "tvlInUsd": "70585310.75762358"
        }
    },
    "updatedAt": "2024-11-19T21:15:41.097Z"
}
Data saved to aa

In [3]:
# 2. Protocol Liquidity V1-Here we get HTML!
def get_protocol_liquidity_v1(pool_id, date):
    url = f"{BASE_URL}/data/liquidity/v1"
    params = {
        "poolId": pool_id,
        "date": date  # Use MM-DD-YYYY format
    }
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        print(f"Request URL: {response.url}")
        
        if response.status_code == 200:
            try:
                data = response.json()
                print("Protocol Liquidity V1 Data:")
                print(json.dumps(data, indent=4))
                flat_data = flatten_dict(data)
                save_to_csv([flat_data], "aave_liquidity_v1.csv")
            except json.JSONDecodeError:
                print("Error: Response is not in JSON format.")
                print("Raw Response Text:", response.text)
        else:
            print(f"Error fetching liquidity v1: {response.status_code}")
            print(f"Details: {response.text}")
    except Exception as e:
        print(f"Failed to fetch protocol liquidity v1: {e}")
        
        
get_protocol_liquidity_v1("0x24a42fd28c976a61df5d00d0599c34c4f90748c8", "01-01-2020")

Request URL: https://aave-api-v2.aave.com/data/liquidity/v1?poolId=0x24a42fd28c976a61df5d00d0599c34c4f90748c8&date=01-01-2020
Error: Response is not in JSON format.
Raw Response Text: 
<!-- HTML for static distribution bundle build -->
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Aave API Documentation</title>
  <link rel="stylesheet" type="text/css" href="./swagger-ui.css" >
  <link rel="icon" href="/assets/favicon.ico" />
  
  <style>
    html
    {
      box-sizing: border-box;
      overflow: -moz-scrollbars-vertical;
      overflow-y: scroll;
    }
    *,
    *:before,
    *:after
    {
      box-sizing: inherit;
    }

    body {
      margin:0;
      background: #fafafa;
    }
  </style>
</head>

<body>

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" style="position:absolute;width:0;height:0">
  <defs>
    <symbol viewBox="0 0 20 20" id="unlocked">
      <path d="M15.8 8H14V5.6C14 2.703 12.665 1 10 1 7.334 1 6 2.70

In [4]:
# 3. Protocol Liquidity V2- Here we get HTML!
def get_protocol_liquidity_v2(pool_id, date):
    url = f"{BASE_URL}/data/liquidity/v2"
    params = {
        "poolId": pool_id,
        "date": date  # Use MM-DD-YYYY format
    }
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        print(f"Request URL: {response.url}")
        
        if response.status_code == 200:
            try:
                data = response.json()
                print("Protocol Liquidity V2 Data:")
                print(json.dumps(data, indent=4))
                flat_data = flatten_dict(data)
                save_to_csv([flat_data], "aave_liquidity_v2.csv")
            except json.JSONDecodeError:
                print("Error: Response is not in JSON format.")
                print("Raw Response Text:", response.text)
        else:
            print(f"Error fetching liquidity v2: {response.status_code}")
            print(f"Details: {response.text}")
    except Exception as e:
        print(f"Failed to fetch protocol liquidity v2: {e}")
        
get_protocol_liquidity_v2("0xb53c1a33016b2dc2ff3653530bff1848a515c8c5", "01-01-2020")

Request URL: https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date=01-01-2020
Error: Response is not in JSON format.
Raw Response Text: 
<!-- HTML for static distribution bundle build -->
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Aave API Documentation</title>
  <link rel="stylesheet" type="text/css" href="./swagger-ui.css" >
  <link rel="icon" href="/assets/favicon.ico" />
  
  <style>
    html
    {
      box-sizing: border-box;
      overflow: -moz-scrollbars-vertical;
      overflow-y: scroll;
    }
    *,
    *:before,
    *:after
    {
      box-sizing: inherit;
    }

    body {
      margin:0;
      background: #fafafa;
    }
  </style>
</head>

<body>

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" style="position:absolute;width:0;height:0">
  <defs>
    <symbol viewBox="0 0 20 20" id="unlocked">
      <path d="M15.8 8H14V5.6C14 2.703 12.665 1 10 1 7.334 1 6 2.70

In [59]:
# 4. Pool Data - staking pool (stkAAVE, stkABPT) stats
def get_pools_data():
    url = f"{BASE_URL}/data/pools"
    try:
        response = requests.get(url, headers=HEADERS)
        print(f"Request URL: {response.url}")
        
        if response.status_code == 200:
            data = response.json()
            print("Aave Pools Data:")
            print(json.dumps(data, indent=4))
            flat_data = [flatten_dict(pool) for pool in data]
            save_to_csv(flat_data, "aave_pools_data.csv")
        else:
            print(f"Error fetching pool data: {response.status_code}")
            print(f"Details: {response.text}")
    except Exception as e:
        print(f"Failed to fetch pools data: {e}")

# Fetch and save the data
get_pools_data()

Request URL: https://aave-api-v2.aave.com/data/pools
Aave Pools Data:
[
    {
        "liquidity": {
            "usd": 535155486.3432173,
            "eth": 257943.89914414583,
            "native": 2805454.0170810767
        },
        "price": {
            "eth": 0.0919437273160237,
            "usd": 190.7553939878928
        },
        "address": "0x4da27a545c0c5B758a6BA100e3a049001de870f5",
        "apy": 4.68,
        "name": "Staked Aave",
        "symbol": "stkAAVE",
        "updatedAt": "2025-03-25T17:15:37.590Z"
    },
    {
        "liquidity": {
            "usd": 989886.499145087,
            "eth": 477.1233591275843,
            "native": 4149872.2168122954
        },
        "price": {
            "eth": 0.000114973024276416,
            "usd": 0.2385342120017044
        },
        "address": "0xa1116930326D21fB917d5A27F1E9943A9595fb47",
        "apy": 0,
        "name": "Staked Aave Balancer Pool Token",
        "symbol": "stkABPT",
        "updatedAt": "2025-03-25T17

## Compound
We fetch
- MarketHistoryService

API documentation: https://stage.compound.finance/docs/api
The API was shut down probably. We need to try other ways. I try Etherscan and Infura.

https://github.com/compound-finance/compound-protocol

https://www.comp.xyz/t/compound-v2-rest-api-permanent-shut-down/4143


In [ ]:
# cToken borrow rate and supply rate 

import requests
import json
from web3 import Web3

# Input your API Key and Infura URL
api_key = input('Enter your Etherscan API Key: ')
infura_url = input('Enter your Infura URL: ')

# Connect to Ethereum Mainnet via Infura
w3 = Web3(Web3.HTTPProvider(infura_url))

# Compound Unitroller (proxy for Comptroller) contract address
unitroller_address = w3.to_checksum_address('0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b')


def get_contract_abi(address):
    try:
        url = f'https://api.etherscan.io/api?module=contract&action=getabi&address={address}&apikey={api_key}'
        response = requests.get(url)
        if response.status_code == 200:
            abi = response.json()
            if abi['status'] == '1':
                return json.loads(abi['result'])
            else:
                print(f"Error fetching ABI: {abi['message']}")
                return None
        else:
            print(f'Failed to retrieve ABI: {response.status_code}')
            return None
    except Exception as e:
        print(f'Error fetching ABI: {str(e)}')
        return None


def get_implementation_address(unitroller):
    try:
        # Fetching the current implementation address using the Comptroller proxy
        unitroller_contract = w3.eth.contract(address=unitroller, abi=get_contract_abi(unitroller))
        implementation_address = unitroller_contract.functions.comptrollerImplementation().call()
        return w3.to_checksum_address(implementation_address)
    except Exception as e:
        print(f'Error fetching implementation address: {str(e)}')
        return None


def fetch_cToken_borrow_supply():
    try:
        implementation_address = get_implementation_address(unitroller_address)
        if not implementation_address:
            print('Unable to fetch implementation address')
            return

        abi = get_contract_abi(implementation_address)
        if not abi:
            print('Unable to fetch ABI')
            return

        unitroller = w3.eth.contract(address=unitroller_address, abi=abi)

        # Fetching the list of all cTokens through the Unitroller
        ctoken_addresses = unitroller.functions.getAllMarkets().call()
        print(f'Available cTokens: {ctoken_addresses}')

        for ctoken_address in ctoken_addresses:
            ctoken_abi = get_contract_abi(ctoken_address)
            if not ctoken_abi:
                continue
            ctoken = w3.eth.contract(address=w3.to_checksum_address(ctoken_address), abi=ctoken_abi)
            symbol = ctoken.functions.symbol().call()
            borrow_rate = ctoken.functions.borrowRatePerBlock().call()
            supply_rate = ctoken.functions.supplyRatePerBlock().call()
            print(f'{symbol} - Borrow Rate: {borrow_rate}, Supply Rate: {supply_rate}')

    except Exception as e:
        print(f'Failed to fetch Compound market history data: {str(e)}')


# Example usage
fetch_cToken_borrow_supply()


Available cTokens: ['0x6C8c6b02E7b2BE14d4fA6022Dfd6d75921D90E4E', '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643', '0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5', '0x158079Ee67Fce2f58472A96584A73C7Ab9AC95c1', '0x39AA39c021dfbaE8faC545936693aC917d5E7563', '0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9', '0xC11b1268C1A384e55C48c2391d8d480264A3A7F4', '0xB3319f5D18Bc0D84dD1b4825Dcde5d5f7266d407', '0xF5DCe57282A584D2746FaF1593d3121Fcac444dC', '0x35A18000230DA775CAc24873d00Ff85BccdeD550', '0x70e36f6BF80a52b3B46b3aF8e106CC0ed743E8e4', '0xccF4429DB6322D5C611ee964527D42E5d685DD6a', '0x12392F67bdf24faE0AF363c24aC620a2f67DAd86', '0xFAce851a4921ce59e912d19329929CE6da6EB0c7', '0x95b4eF2869eBD94BEb4eEE400a99824BF5DC325b', '0x4B0181102A0112A2ef11AbEE5563bb4a3176c9d7', '0xe65cdB6479BaC1e22340E4E755fAE7E509EcD06c', '0x80a2AE356fc9ef4305676f7a3E2Ed04e12C33946', '0x041171993284df560249B57358F931D9eB7b925D', '0x7713DD9Ca933848F6819F38B8352D9A15EA73F67']
cBAT - Borrow Rate: 9663855520, Supply Rate: 5221597
cDAI - 

In [ ]:
# this seems to give you the same thing... 
import requests
import json
from web3 import Web3
import time

# Input your Infura URL
infura_url = input('Enter your Infura URL: ')

# Connect to Ethereum Mainnet via Infura
w3 = Web3(Web3.HTTPProvider(infura_url))

# Compound Unitroller (proxy for Comptroller) contract address
unitroller_address = Web3.to_checksum_address('0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b')


def get_historical_market_data(asset, start_time, end_time, num_buckets):
    try:
        # Use Infura to interact directly with blockchain contracts
        ctoken = w3.eth.contract(address=Web3.to_checksum_address(asset), abi=[])
        print(f'Fetching historical data for asset: {asset}')

        # Example call to get current block number
        current_block = w3.eth.block_number
        print(f'Current Ethereum Block Number: {current_block}')

        # Sample data simulation (since Infura directly may not give historical rates)
        print(f'Historical Data for Asset {asset}:')
        for i in range(num_buckets):
            print(f'Block Number: {current_block - i} - Simulated Supply Rate: {0.02 * (i + 1)}')
            print(f'Block Number: {current_block - i} - Simulated Borrow Rate: {0.01 * (i + 1)}')
    except Exception as e:
        print(f'Error fetching historical data: {str(e)}')


def fetch_cToken_borrow_supply():
    try:
        # Example: List of cToken addresses to fetch rates
        ctoken_addresses = [
            '0x39AA39c021dfbaE8faC545936693aC917d5E7563',  # cUSDC
            '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643'   # cDAI
        ]

        for ctoken_address in ctoken_addresses:
            ctoken = w3.eth.contract(address=w3.to_checksum_address(ctoken_address), abi=[])
            symbol = ctoken.functions.symbol().call()
            borrow_rate = ctoken.functions.borrowRatePerBlock().call()
            supply_rate = ctoken.functions.supplyRatePerBlock().call()
            print(f'{symbol} - Borrow Rate: {borrow_rate}, Supply Rate: {supply_rate}')

    except Exception as e:
        print(f'Failed to fetch cToken rates: {str(e)}')


def fetch_market_history():
    try:
        asset = input('Enter the asset address: ')
        num_buckets = int(input('Enter the number of buckets to group data: '))
        get_historical_market_data(asset, 0, 0, num_buckets) # fetch the latest available data 
    except Exception as e:
        print(f'Failed to fetch Compound market history data: {str(e)}')


# Example usage
fetch_market_history()


Fetching historical data for asset: 0x39AA39c021dfbaE8faC545936693aC917d5E7563
Current Ethereum Block Number: 22154672
Historical Data for Asset 0x39AA39c021dfbaE8faC545936693aC917d5E7563:
Block Number: 22154672 - Simulated Supply Rate: 0.02
Block Number: 22154672 - Simulated Borrow Rate: 0.01
Block Number: 22154671 - Simulated Supply Rate: 0.04
Block Number: 22154671 - Simulated Borrow Rate: 0.02
Block Number: 22154670 - Simulated Supply Rate: 0.06
Block Number: 22154670 - Simulated Borrow Rate: 0.03
Block Number: 22154669 - Simulated Supply Rate: 0.08
Block Number: 22154669 - Simulated Borrow Rate: 0.04
Block Number: 22154668 - Simulated Supply Rate: 0.1
Block Number: 22154668 - Simulated Borrow Rate: 0.05
Failed to fetch cToken rates: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')


## Lendle
Lendle does not offer a public API to directly fetch historical market data like the Compound V2 API did.
 

I use this to get ABI and address, but I did not get anything lol
https://explorer.mantle.xyz/address/0x13e9761c037f382472cE765556c3dA2aF29d9EC7?tab=contract

Documentation:
https://www.lendle.xyz/
https://v2-lendle.xyz/

In [63]:
pip install web3

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 24.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/587.0 kB ? eta -:--:--
   --------------------------------------- 587.0/587.0 kB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 32.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from web3 import Web3

# Connect to Mantle Network via a public RPC endpoint
mantle_rpc = 'https://rpc.mantle.xyz'
web3 = Web3(Web3.HTTPProvider(mantle_rpc))

if web3.is_connected():
    print("Connected to Mantle Network")
else:
    print("Failed to connect to Mantle Network")

# Replace with the actual contract address and ABI from Lendle documentation
contract_address = '0x13e9761c037f382472cE765556c3dA2aF29d9EC7'
abi = [
    {
        "inputs": [],
        "name": "FLASHLOAN_PREMIUM_TOTAL",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "LENDINGPOOL_REVISION",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "MAX_NUMBER_RESERVES",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "MAX_STABLE_RATE_BORROW_SIZE_PERCENT",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "asset",
                "type": "address"
            },
            {
                "internalType": "uint256",
                "name": "amount",
                "type": "uint256"
            },
            {
                "internalType": "uint256",
                "name": "interestRateMode",
                "type": "uint256"
            },
            {
                "internalType": "uint16",
                "name": "referralCode",
                "type": "uint16"
            },
            {
                "internalType": "address",
                "name": "onBehalfOf",
                "type": "address"
            }
        ],
        "name": "borrow",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "asset",
                "type": "address"
            },
            {
                "internalType": "uint256",
                "name": "amount",
                "type": "uint256"
            },
            {
                "internalType": "address",
                "name": "onBehalfOf",
                "type": "address"
            },
            {
                "internalType": "uint16",
                "name": "referralCode",
                "type": "uint16"
            }
        ],
        "name": "deposit",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "name": "reserve",
                "type": "address"
            },
            {
                "indexed": False,
                "name": "user",
                "type": "address"
            },
            {
                "indexed": True,
                "name": "onBehalfOf",
                "type": "address"
            },
            {
                "indexed": False,
                "name": "amount",
                "type": "uint256"
            },
            {
                "indexed": True,
                "name": "referral",
                "type": "uint16"
            }
        ],
        "name": "Deposit",
        "type": "event"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "name": "collateralAsset",
                "type": "address"
            },
            {
                "indexed": True,
                "name": "debtAsset",
                "type": "address"
            },
            {
                "indexed": True,
                "name": "user",
                "type": "address"
            },
            {
                "indexed": False,
                "name": "debtToCover",
                "type": "uint256"
            },
            {
                "indexed": False,
                "name": "liquidatedCollateralAmount",
                "type": "uint256"
            },
            {
                "indexed": False,
                "name": "liquidator",
                "type": "address"
            },
            {
                "indexed": False,
                "name": "receiveAToken",
                "type": "bool"
            }
        ],
        "name": "LiquidationCall",
        "type": "event"
    },
    {
        "inputs": [],
        "name": "paused",
        "outputs": [
            {
                "internalType": "bool",
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "bool",
                "name": "val",
                "type": "bool"
            }
        ],
        "name": "setPause",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]


# Initialize the contract instance
lendle_contract = web3.eth.contract(address=contract_address, abi=abi)

def get_deposit_events(batch_size=1000):
    try:
        # Get the latest block number
        latest_block = web3.eth.block_number
        start_block = 0  # Start from the first block

        # Correctly calculate the event signature hash with "0x" prefix
        event_signature_hash = "0x" + web3.keccak(text="Deposit(address,address,address,uint256,uint16)").hex()

        while start_block <= latest_block:
            end_block = min(start_block + batch_size, latest_block)

            print(f"Fetching blocks from {start_block} to {end_block}...")

            # Define the filter parameters for the current batch
            filter_params = {
                "fromBlock": start_block,
                "toBlock": end_block,
                "address": contract_address,
                "topics": [event_signature_hash]  # Correctly formatted topic
            }

            try:
                # Fetch logs for the current batch
                logs = web3.eth.get_logs(filter_params)

                # Decode the logs
                for log in logs:
                    decoded_event = lendle_contract.events.Deposit().processLog(log)
                    print(f"Deposit Event: {decoded_event['args']}")
            
            except Exception as e:
                print(f"Error fetching logs from block {start_block} to {end_block}: {e}")

            # Move to the next batch
            start_block = end_block + 1

    except Exception as e:
        print("Error fetching deposit events from Lendle contract:", e)

# Fetch deposit events with batching
get_deposit_events(batch_size=1000)


Connected to Mantle Network
Fetching blocks from 0 to 1000...
Fetching blocks from 1001 to 2001...
Fetching blocks from 2002 to 3002...
Fetching blocks from 3003 to 4003...
Fetching blocks from 4004 to 5004...
Fetching blocks from 5005 to 6005...
Fetching blocks from 6006 to 7006...
Fetching blocks from 7007 to 8007...
Fetching blocks from 8008 to 9008...
Fetching blocks from 9009 to 10009...
Fetching blocks from 10010 to 11010...
Fetching blocks from 11011 to 12011...
Fetching blocks from 12012 to 13012...
Fetching blocks from 13013 to 14013...
Fetching blocks from 14014 to 15014...
Fetching blocks from 15015 to 16015...
Fetching blocks from 16016 to 17016...
Fetching blocks from 17017 to 18017...
Fetching blocks from 18018 to 19018...
Fetching blocks from 19019 to 20019...
Fetching blocks from 20020 to 21020...
Fetching blocks from 21021 to 22021...
Fetching blocks from 22022 to 23022...
Fetching blocks from 23023 to 24023...
Fetching blocks from 24024 to 25024...
Fetching blocks fro

KeyboardInterrupt: 

# P2: ML - Here is one possible ML model
Data Attributes:
- Protocol: Name of the DeFi protocol, such as Aave, Compound
- APY: Annual Percentage Yield offered by the protocol
- Gas Fee- Random values between 0.001 and 0.02 ETH
- Risk Score: Random values between 0 and 1 (0 = low risk, 1 = high risk)
- Timestamp	Time when data was recorded	2025-03-29 22:44:40

Given a set of features (protocol, gas fee, and risk score), predict the most profitable staking position (highest APY) among various DeFi protocols.


- APY (Annual Percentage Yield): Represents the expected return on investment when staking assets in a DeFi protocol. Higher APY is better.

- Gas Fees: Represents the cost of executing staking or unstaking transactions. Lower gas fees are better.

- Risk Score: Represents the risk associated with the DeFi protocol (e.g., smart contract vulnerabilities, liquidity risks). Lower risk scores are better.



Data Flow
1. Simulated data from DeFi protocols (Aave, Compound, Lendle).

2. Attributes: protocol, apy, gas_fee, risk_score, timestamp.


Feature Set:

1. Input Features: protocol_encoded (one-hot coding), gas_fee (continuous), risk_score (continuous)

2. Target: apy (continuous)


Model Training: RandomForestRegressor
Split the data into training and testing sets (80-20 split). Train the model on the training set. Validate on the testing set to calculate Mean Squared Error (MSE).


Model Prediction Logic:
The model learns how each combination of protocol, gas_fee, and risk_score affects the apy.

Given a new set of inputs (e.g., Compound, 0.0085, 0.74), the model predicts the most likely APY.

The prediction aims to maximize yield while minimizing gas fees and risk.


In [7]:
pip install requests couchdb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -------------------------- ------------- 7.3/11.1 MB 41.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 38.6 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   ------ --------------------------------- 7.1/40.9 MB 36.4 MB/s eta 0:00:01
   --------------- ------------------------ 15.5/40.9 MB 37.4 MB/s eta 0:00:01
   ----------------------- ---------------- 23.9/40.9 MB 39.7 MB/s eta 0:00:01
   ------------------------------- -------- 32.2/40.9 MB 39.4 MB/s eta 0:00:01
   ---------------------------------------  40.9/40.9 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------- 40.9/4


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Step 1: Simulate Data
np.random.seed(42)

protocols = ['Aave', 'Compound', 'Lendle']
num_samples = 1000

data = {
    'protocol': np.random.choice(protocols, num_samples),
    'gas_fee': np.random.uniform(0.001, 0.01, num_samples),  # Gas fee between 0.001 and 0.01
    'risk_score': np.random.uniform(0, 1, num_samples),       # Risk score between 0 and 1
    'apy': np.random.uniform(0.01, 0.2, num_samples),         # APY between 0.01 and 0.2
}

df = pd.DataFrame(data)

# One-hot encode the protocol feature
encoder = OneHotEncoder()
protocol_encoded = encoder.fit_transform(df[['protocol']]).toarray()
encoded_df = pd.DataFrame(protocol_encoded, columns=encoder.get_feature_names_out(['protocol']))

# Combine encoded protocol with other features
df = pd.concat([encoded_df, df[['gas_fee', 'risk_score', 'apy']]], axis=1)

# Step 2: Split the data into training and testing sets (80-20 split)
X = df.drop(columns=['apy'])
y = df['apy']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the RandomForestRegressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 4: Predict on the test set
y_pred = model.predict(X_test)

# Step 5: Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Display the MSE and first few predictions
mse, y_pred[:10], y_test[:10].values


(0.003630810820057357,
 array([0.09860842, 0.10006489, 0.12606582, 0.08972737, 0.14738925,
        0.14489959, 0.07621193, 0.07572881, 0.05271853, 0.10507832]),
 array([0.01100956, 0.06688284, 0.05872798, 0.10506829, 0.15218761,
        0.16924945, 0.1548375 , 0.09820532, 0.04884519, 0.02490661]))

# P3. adjust the ML model based on what I can find...
I want to use Aave's markets data, but API is deprecated!
I can fetch Aave's /data/daily-volume-24-hours.

Adjustment:
I cannot APY data, so I estimate that.

Use priceInEth, borrow, deposit, repay, and withdrawal as input features.

Encode the symbol column using one-hot encoding to include it as a categorical feature and input feature.

Estimated APY (Annual Percentage Yield): The model predicts the estimated APY as a percentage based on the input features.

Train a model to predict estimated APY based on these features.

In [30]:
import requests
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

BASE_URL = "https://aave-api-v2.aave.com"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Save data to CSV
def save_to_csv(data, filename):
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

# Flatten nested dictionary for easier CSV storage
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Preprocess Aave daily volume data for ML model
def preprocess_data(data):
    processed_data = []
    for item in data.get('reserves', {}).get('v2', []):
        flat_item = flatten_dict(item)
        # Calculate estimated APY as a new feature
        try:
            deposit = float(flat_item.get('deposit', 0))
            borrow = float(flat_item.get('borrow', 0))
            estimated_apy = ((borrow - deposit) / deposit) * 100 if deposit != 0 else 0
            flat_item['estimated_apy'] = estimated_apy
        except:
            flat_item['estimated_apy'] = 0
        processed_data.append(flat_item)
    df = pd.DataFrame(processed_data)
    # Select relevant columns
    columns = ['priceInEth', 'borrow', 'deposit', 'repay', 'withdrawal', 'symbol', 'estimated_apy']
    df = df[columns]
    df = df.dropna()
    # One-hot encode the 'symbol' column
    encoder = OneHotEncoder()
    encoded_symbols = encoder.fit_transform(df[['symbol']]).toarray()
    encoded_df = pd.DataFrame(encoded_symbols, columns=encoder.get_feature_names_out(['symbol']))
    df = pd.concat([df.drop(columns=['symbol']), encoded_df], axis=1)
    print("Processed Data Columns:", df.columns)
    save_to_csv(df, "aave_daily_volume_data.csv")
    return df

# Train the model to predict APY
def train_apy_model(df):
    numeric_df = df.select_dtypes(include=[np.number]).dropna()
    print("Numeric Data Columns:", numeric_df.columns)
    if 'estimated_apy' in numeric_df.columns:
        X = numeric_df.drop(columns=['estimated_apy'])
        y = numeric_df['estimated_apy']
    else:
        print("No APY-related column found for training.")
        return

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    return model

# Get Aave Daily Volume Data and preprocess it
def get_daily_volume_data():
    url = f"{BASE_URL}/data/daily-volume-24-hours"
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            print("Aave Daily Volume Data fetched successfully.")
            df = preprocess_data(data)
            train_apy_model(df)
        else:
            print(f"Error fetching Daily Volume Data: {response.status_code}")
            print(f"Details: {response.text}")
    except Exception as e:
        print(f"Failed to fetch Daily Volume Data: {e}")

get_daily_volume_data()


Aave Daily Volume Data fetched successfully.
Processed Data Columns: Index(['priceInEth', 'borrow', 'deposit', 'repay', 'withdrawal',
       'estimated_apy', 'symbol_1INCH', 'symbol_AAVE', 'symbol_AMPL',
       'symbol_AmmBptBALWETH', 'symbol_AmmBptWBTCWETH', 'symbol_AmmDAI',
       'symbol_AmmGUniDAIUSDC', 'symbol_AmmGUniUSDCUSDT', 'symbol_AmmUSDC',
       'symbol_AmmUSDT', 'symbol_AmmUniAAVEWETH', 'symbol_AmmUniBATWETH',
       'symbol_AmmUniCRVWETH', 'symbol_AmmUniDAIUSDC', 'symbol_AmmUniDAIWETH',
       'symbol_AmmUniLINKWETH', 'symbol_AmmUniMKRWETH', 'symbol_AmmUniRENWETH',
       'symbol_AmmUniSNXWETH', 'symbol_AmmUniUNIWETH', 'symbol_AmmUniUSDCWETH',
       'symbol_AmmUniWBTCUSDC', 'symbol_AmmUniWBTCWETH',
       'symbol_AmmUniYFIWETH', 'symbol_AmmWBTC', 'symbol_AmmWETH',
       'symbol_BAL', 'symbol_BAT', 'symbol_BUSD', 'symbol_CRV', 'symbol_CVX',
       'symbol_DAI', 'symbol_DPI', 'symbol_ENJ', 'symbol_ENS', 'symbol_FEI',
       'symbol_FRAX', 'symbol_GUSD', 'symbol_KNC', 'sym